In [1]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2

In [2]:
import numpy as np
import argparse
import pickle
import cv2
import os
import time 
from keras.models import load_model
from collections import deque

def print_results(video, limit=None):
    #fig=plt.figure(figsize=(16, 30))
    if not os.path.exists('Output_V'):
        os.mkdir('Output_V')
        
    if not os.path.exists('Output_NV'):
        os.mkdir('Output_NV')

    print("Loading model ...")
    model = load_model('modelnew.h5')
    Q = deque(maxlen=128)
    vs = cv2.VideoCapture(video)
    writer = None
    (W, H) = (None, None)
    count = 0

    # Get the current timestamp
    timestamp = int(time.time())

    while True:
        # read the next frame from the file
        (grabbed, frame) = vs.read()

        # if the frame was not grabbed, then we have reached the end
        # of the stream
        if not grabbed:
            break

        # if the frame dimensions are empty, grab them
        if W is None or H is None:
            (H, W) = frame.shape[:2]

        # clone the output frame, then convert it from BGR to RGB
        # ordering, resize the frame to a fixed 128x128, and then
        # perform mean subtraction
        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        # make predictions on the frame and then update the predictions
        # queue
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        # perform prediction averaging over the current history of
        # previous predictions
        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]
        label = i

        text_color = (0, 255, 0) # default : green

        if label: # Violence prob
            text_color = (0, 0, 255) # red
        else:
            text_color = (0, 255, 0)

        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX 

        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3) 

        # check if the video writer is None
        if writer is None:
            # initialize our video writer
            if label==True:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                output_filename = "Output_V/v_output_{}.avi".format(timestamp)
                writer = cv2.VideoWriter(output_filename, fourcc, 30, (W, H), True)
            else:
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                output_filename = "Output_NV/v_output_{}.avi".format(timestamp)
                writer = cv2.VideoWriter(output_filename, fourcc, 30, (W, H), True)

        # write the output frame to disk
        writer.write(output)

        # show the output image
        cv2.imshow("Output", output)
        key = cv2.waitKey(10) & 0xFF

        # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
            
    writer.release()
    vs.release()
    cv2.destroyAllWindows()



In [3]:
print_results(r'C:\Users\siddh\Desktop\Sidd_final_yr_project\Violence-Alert-System\Violence Detection\Testing videos\V_19.mp4')

Loading model ...
1/1 [==============================] - 0s 62ms/step


In [4]:
print_results(r'C:\Users\siddh\Desktop\Sidd_final_yr_project\Violence-Alert-System\Violence Detection\Testing videos\nonv.mp4')

Loading model ...
1/1 [==============================] - 0s 70ms/step


In [35]:
import os
print(os.getcwd())

os.chdir("../video_creation")
print(os.getcwd())

C:\Users\siddh\Desktop\Sidd_final_yr_project\Violence-Alert-System\video_creation
C:\Users\siddh\Desktop\Sidd_final_yr_project\Violence-Alert-System\video_creation


In [36]:
%run Capture_Alert.ipynb

In [26]:
os.chdir("../violence Detection")
print(os.getcwd())

C:\Users\siddh\Desktop\Sidd_final_yr_project\Violence-Alert-System\violence Detection


In [5]:
print_results('capture.mp4')

Loading model ...
1/1 [==============================] - 0s 34ms/step
